In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
os.environ['NEO4J_URI'] = os.getenv("NEO4J_URI")  ###Used for LangSmith tracking
os.environ['NEO4J_USERNAME'] = os.getenv("NEO4J_USERNAME")
os.environ['NEO4J_PASSWORD'] = os.getenv("NEO4J_PASSWORD")

In [3]:
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")  
llm = ChatGroq(groq_api_key=groq_api_key, model_name = "Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021CBF38D760>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021CBEFF7980>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [5]:
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph()

In [6]:
movie_query = """
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' AS row

MERGE (m:Movie {id: row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)

FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name: trim(director)})
    MERGE (p)-[:DIRECTED]->(m))

FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name: trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))

FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name: trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))



"""

In [7]:
graph.query(movie_query)

[]

In [10]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(graph=graph,llm=llm, exclude_types=["Genre"], verbose=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x0000021CBF39D340>, cypher_generation_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'schema'], template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021CBF38D760>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021CBEFF7980>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [11]:
chain.schema

<bound method BaseModel.schema of <class 'langchain_community.chains.graph_qa.cypher.GraphCypherQAChain'>>

In [12]:
chain.graph_schema

'Node properties are the following:\nCEO {name: STRING, YOB: INTEGER},Company {nmae: STRING},Student {name: STRING, YOB: INTEGER, POB: STRING},Country {name: STRING},User {name: STRING, city: STRING, userId: INTEGER, age: INTEGER},Post {postId: INTEGER, content: STRING, timestamp: DATE_TIME},Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT},Person {name: STRING}\nRelationship properties are the following:\n\nThe relationships are the following:\n(:User)-[:POSTED]->(:Post),(:User)-[:LIKES]->(:User),(:User)-[:FRIEND]->(:User),(:Person)-[:DIRECTED]->(:Movie),(:Person)-[:ACTED_IN]->(:Movie)'